In [ ]:
from scenarios.dependency_injection import injector
from arbeitszeit.use_cases import get_statistics, register_hours_worked, get_member_account, query_plans, show_my_accounts
from tests.data_generators import MemberGenerator, CompanyGenerator, PlanGenerator, CooperationGenerator, ConsumptionGenerator
from arbeitszeit.datetime_service import DatetimeService
from arbeitszeit.records import ProductionCosts
from datetime import datetime
from pprint import pprint
from decimal import Decimal

# Scenario

#### Persons: 

- E1, E2: Europroducers (wage labourers)
- A1: Graphic Designer
- A2: Baker

## January 2, 2024

E1 and E2 meet in the pub and, after two beers, decide to start an instance of the working time app. They want to feed €250 of their wages into the network each month as euro producers. The aim is to grow the labor time accounting economy. Every hour should count for the same amount in the network, resulting in an average remuneration. They agree to meet online every month to decide on the general rules of the network.

In [ ]:
datetime_service = injector.get(DatetimeService)

datetime_service.freeze_time(datetime(2024, 1, 2))
print(f"Current date: {datetime_service.now()}")

## January 3, 2024

In [ ]:
datetime_service.freeze_time(datetime(2024, 1, 3))
print(f"Current date: {datetime_service.now()}")


Both register as workers first, then each of them registers a euro producer company. Finally, they add themselves as workers to their company. 

In [ ]:
member_generator = injector.get(MemberGenerator)
company_generator = injector.get(CompanyGenerator)

e1_worker = {"email":"e1_worker@mail.org", "name":"E1 Worker", "password":"safe"}
e1_worker_id = member_generator.create_member(email=e1_worker["email"], name=e1_worker["name"], password=e1_worker["password"])
e2_worker = {"email":"e2_worker@mail.org", "name":"E2 Worker", "password":"safe"}
e2_worker_id = member_generator.create_member(email=e2_worker["email"], name=e2_worker["name"], password=e2_worker["password"])

e1_company = {"email":"e1_company@mail.org", "name":"E1 Company", "password":"safe"}
e1_company_id = company_generator.create_company(email=e1_company["email"], name=e1_company["name"], password=e1_company["password"], workers=[e1_worker_id])
e2_company = {"email":"e2_company@mail.org", "name":"E2 Company", "password":"safe"}
e2_company_id = company_generator.create_company(email=e2_company["email"], name=e2_company["name"], password=e2_company["password"], workers=[e2_worker_id])

The app's statistics now show 2 registered companies and 2 registered workers:

In [ ]:
get_statistics_use_case = injector.get(get_statistics.GetStatistics)
get_statistics_response = get_statistics_use_case()
print(f"Registered companies: {get_statistics_response.registered_companies_count}")
print(f"Registered members: {get_statistics_response.registered_members_count}")

## January 5, 2024

In [ ]:
datetime_service.freeze_time(datetime(2024, 1, 5))
print(f"Current date: {datetime_service.now()}")


Two days later, both workers create a plan each in which they state that they want to produce 250 euros each in the upcoming month. E1 has a higher hourly wage (25€/hour) than E2 (10€/hour). This means, E2 has to work longer to produce 250€, his labour costs are higher.

In [ ]:
plan_generator = injector.get(PlanGenerator)
plan_e1 = plan_generator.create_plan(
    planner=e1_company_id,
    amount = 250,
    approved = True,
    costs = ProductionCosts(resource_cost=Decimal(0), means_cost=Decimal(0), labour_cost=Decimal(10)),
    product_name="Euros for the network.",
    production_unit="1 Euro",
    timeframe=30,
)
print(f"Plan created: '{plan_e1.prd_name}'")

plan_e2 = plan_generator.create_plan(
    planner=e2_company_id,
    amount = 250,
    approved = True,
    costs = ProductionCosts(resource_cost=Decimal(0), means_cost=Decimal(0), labour_cost=Decimal(25)),
    product_name="Euros for the network.",
    production_unit="1 Euro",
    timeframe=30,
)
print(f"Plan created: '{plan_e2.prd_name}'")

The euros appear in the list of available products:

In [ ]:
query_plans_use_case = injector.get(query_plans.QueryPlans)
query_plans_response = query_plans_use_case(request=query_plans.QueryPlansRequest(query_string="", filter_category=query_plans.PlanFilter.by_plan_id, sorting_category=query_plans.PlanSorting.by_company_name))
products = [print(f"'{plan.product_name}', by {plan.company_name}, Costs {round(plan.price_per_unit, 2)}") for plan in query_plans_response.results]

E1 and E2 register their hours worked and get remunerated with 10 and 25 work certificates respectively:

In [ ]:
register_hours_worked_use_case = injector.get(register_hours_worked.RegisterHoursWorked)
register_hours_worked_use_case(
    use_case_request=register_hours_worked.RegisterHoursWorkedRequest(
        company_id=e1_company_id,
        worker_id=e1_worker_id,
        hours_worked=10,
    )
)
register_hours_worked_use_case(
    use_case_request=register_hours_worked.RegisterHoursWorkedRequest(
        company_id=e2_company_id,
        worker_id=e2_worker_id,
        hours_worked=25,
    )
)

get_member_account_use_case = injector.get(get_member_account.GetMemberAccount)
get_member_account_response1 = get_member_account_use_case(
    member_id=e1_worker_id,
)
get_member_account_response2 = get_member_account_use_case(
    member_id=e2_worker_id,
)
print(f"Work certificates of E1: {get_member_account_response1.balance}")
print(f"Work certificates of E2: {get_member_account_response2.balance}")

Both think it's great that they get remunerated for the hours actually worked. It doesn't matter anymore that they got different wages: For every hour worked they get 1 work certificate. 

On the other hand, they think it's unfair that the euros produced by E1 are "cheaper". Probably no one will spend their work certificates on E2's euros.

So they create a cooperative, the cooperative of euro producers.

In [ ]:
cooperation_generator = injector.get(CooperationGenerator)
cooperation_generator.create_cooperation(
    name="Cooperation of Europroducers",
    coordinator=e1_company_id,
    plans=[plan_e1, plan_e2],
)
print()

As a result, now one euro now "costs" the same amount of work in the network.

In [ ]:
query_plans_use_case = injector.get(query_plans.QueryPlans)
query_plans_response = query_plans_use_case(request=query_plans.QueryPlansRequest(query_string="", filter_category=query_plans.PlanFilter.by_plan_id, sorting_category=query_plans.PlanSorting.by_company_name))
products = [print(f"'{plan.product_name}', by {plan.company_name}, Costs {round(plan.price_per_unit, 2)}") for plan in query_plans_response.results]

At this point, there are two plans and one cooperation in the economy:

In [ ]:
get_statistics_use_case = injector.get(get_statistics.GetStatistics)
get_statistics_response = get_statistics_use_case()
print(f"Active plans: {get_statistics_response.active_plans_count}")
print(f"Cooperations: {get_statistics_response.cooperations_count}")

## January 10, 2024

In [ ]:
datetime_service.freeze_time(datetime(2024, 1, 10))
print(f"Current date: {datetime_service.now()}")

A1 is a friend of E1 and E2 and a grafic designer. She offers to design an advertising flyer for the labour time accounting economy. She agrees to receive her emuneration through the labour time network.

She estimates that she will have to work 20 hours for the flyer. 

She registers a worker and a company account and creates the plan.

In [ ]:
a1_member_id = member_generator.create_member(name="A1 Worker", email="a1_worker@mail.org", password="safe")
a1_company_id = company_generator.create_company(name="A1 Company", email="a1_company@mail.org", password="safe", workers=[a1_member_id])

plan_a1 = plan_generator.create_plan(
    planner=a1_company_id,
    amount = 1,
    approved = True,
    costs = ProductionCosts(resource_cost=Decimal(0), means_cost=Decimal(0), labour_cost=Decimal(20)),
    product_name="Flyer design for the network.",
    production_unit="1 Flyer design",
    timeframe=10,
)
print(f"Plan created: '{plan_a1.prd_name}'")

The Flyer appears in the list of available products:

In [ ]:
query_plans_use_case = injector.get(query_plans.QueryPlans)
query_plans_response = query_plans_use_case(request=query_plans.QueryPlansRequest(query_string="", filter_category=query_plans.PlanFilter.by_plan_id, sorting_category=query_plans.PlanSorting.by_company_name))
products = [print(f"'{plan.product_name}', by {plan.company_name}, Costs {round(plan.price_per_unit, 2)}") for plan in query_plans_response.results]

## January 18, 2024

In [ ]:
datetime_service.freeze_time(datetime(2024, 1, 18))
print(f"Current date: {datetime_service.now()}")

A1 finished designing the flyer. She registers 20 hours worked in order to receive certificates.

In [ ]:
register_hours_worked_use_case(
    use_case_request=register_hours_worked.RegisterHoursWorkedRequest(
        company_id=a1_company_id,
        worker_id=a1_member_id,
        hours_worked=20,
    )
)
print()

She spends her 20 work certificates to receive euros from E1. Each euro costs 0.07 hours, so she can "buy" 285 Euros with 20 certificates.

In [ ]:
consumption_generator = injector.get(ConsumptionGenerator)
consumption_generator.create_private_consumption(
    consumer=a1_member_id,
    amount=285,
    plan=plan_e1.id,
)
print()

On the other hand, E1 spends his work cerificates to "pay" the flyer design.

In [ ]:
consumption_generator.create_private_consumption(
    consumer=e1_worker_id,
    amount=1,
    plan=plan_a1.id,
)
print()

The app's statistics now show 3 members, companies and plans:

In [ ]:
get_statistics_response = get_statistics_use_case()
pprint(get_statistics_response)

## February 1st, 2024

In [ ]:
datetime_service.freeze_time(datetime(2024, 2, 1))
print(f"Current date: {datetime_service.now()}")

The monthly online meeting takes place. All three workers are invited. They are having a look at their accounts.

In [ ]:
get_member_account_response1 = get_member_account_use_case(
    member_id=e1_worker_id,
)
get_member_account_response2 = get_member_account_use_case(
    member_id=e2_worker_id,
)
get_member_account_response3 = get_member_account_use_case(
    member_id=a1_member_id,
)
print(f"Work certificates of E1: {get_member_account_response1.balance}")
print(f"Work certificates of E2: {get_member_account_response2.balance}")
print(f"Work certificates of A1: {round(get_member_account_response3.balance, 2)}")

show_my_accounts_use_case = injector.get(show_my_accounts.ShowMyAccounts)

show_e1_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=e1_company_id,
)
)
e1_p, e1_r, e1_a, e1_prd = show_e1_accounts_response.balances
print(f"Account balances of E1: {e1_p} (p), {e1_r} (r), {e1_a} (a), {e1_prd} (prd)")

show_e2_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=e2_company_id,
)
)
e2_p, e2_r, e2_a, e2_prd = show_e2_accounts_response.balances
print(f"Account balances of E2: {e2_p} (p), {e2_r} (r), {e2_a} (a), {e2_prd} (prd)")

show_a1_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=a1_company_id,
)
)
a1_p, a1_r, a1_a, a1_prd = show_a1_accounts_response.balances
print(f"Account balances of A1: {a1_p} (p), {a1_r} (r), {a1_a} (a), {a1_prd} (prd)")

The three workers notice, that the euro producers have not gotten rid of all their euros: E2 has spend 25 hours less than expected (prd=-25). E1 spent some more Euros than expected (prd=+1,40). 

They decide that, in general, more euros should be spent.
E1 and E2 decide not to file new euro producer plans, until there is more demand.

They notice also, that E1 has overdrawn his worker account, while E2 has not spent any of his certificates. They decide that the next project should be mainly "paid" by E2. 


## February 3, 2024 

In [ ]:
datetime_service.freeze_time(datetime(2024, 2, 3))
print(f"Current date: {datetime_service.now()}")

A new worker, A2, joins the economy. He uses 70 Euros and 20 hours of his working time to bake 15 breads in the upcoming month.

Currently, one euro costs 0,07 hours, so A2 has to plan 4,9 hours as ressource costs to be able to spend 70 Euros on his means of production.

In [ ]:
a2_member_id = member_generator.create_member(name="A2 Worker", email="a2_worker@mail.org", password="safe")
a2_company_id = company_generator.create_company(name="A2 Company", email="a2_company@mail.org", password="safe", workers=[a2_member_id])

plan_a2 = plan_generator.create_plan(
    planner=a2_company_id,
    amount = 15,
    approved = True,
    costs = ProductionCosts(resource_cost=Decimal("4.9"), means_cost=Decimal(0), labour_cost=Decimal(20)),
    product_name="Bread.",
    production_unit="1 piece",
    timeframe=30,
)
print(f"Plan created: '{plan_a2.prd_name}'")

The bread appears in the list of available products:

In [ ]:
query_plans_response = query_plans_use_case(request=query_plans.QueryPlansRequest(query_string="", filter_category=query_plans.PlanFilter.by_plan_id, sorting_category=query_plans.PlanSorting.by_company_name))
products = [print(f"'{plan.product_name}', Costs {round(plan.price_per_unit, 2)}") for plan in query_plans_response.results]

A2 receives 70€ for resources/means of production from E2 and registers this transfer. He starts to produce the bread.

In [ ]:
consumption_generator.create_resource_consumption_by_company(
    consumer=a2_company_id,
    amount=70,
    plan=plan_e2.id,
)
print()

Moreover, because the baker A2 does not find useful products in the labour time economy, he decides to use his 20 work certificates to "buy" euros from E2.
If each euro costs 0,07 hours, he can get 285€.

In [ ]:
register_hours_worked_use_case(
    use_case_request=register_hours_worked.RegisterHoursWorkedRequest(
        company_id=a2_company_id,
        worker_id=a2_member_id,
        hours_worked=20,
    )
)
consumption_generator.create_private_consumption(
    consumer=a2_member_id,
    amount=285,
    plan=plan_e2.id,
)
print()

## February 10, 2024

In [ ]:
datetime_service.freeze_time(datetime(2024, 2, 10))
print(f"Current date: {datetime_service.now()}")

E2 uses his work certificates to "buy" all 15 breads from A2 

In [ ]:
consumption_generator.create_private_consumption(
    consumer=e2_worker_id,
    amount=15,
    plan=plan_a2.id,
)
print()

## March 1, 2024

All four workers are invited for the monthly online meeting. 
Again, they are reviewing the accounts.

In [ ]:
get_member_account_response1 = get_member_account_use_case(
    member_id=e1_worker_id,
)
get_member_account_response2 = get_member_account_use_case(
    member_id=e2_worker_id,
)
get_member_account_response3 = get_member_account_use_case(
    member_id=a1_member_id,
)
get_member_account_response4 = get_member_account_use_case(
    member_id=a2_member_id,
)
print(f"Work certificates of E1: {get_member_account_response1.balance}")
print(f"Work certificates of E2: {get_member_account_response2.balance}")
print(f"Work certificates of A1: {round(get_member_account_response3.balance, 2)}")
print(f"Work certificates of A2: {round(get_member_account_response4.balance, 2)}")

show_e1_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=e1_company_id,
)
)
e1_p, e1_r, e1_a, e1_prd = show_e1_accounts_response.balances
print(f"Account balances of E1: {e1_p} (p), {e1_r} (r), {e1_a} (a), {e1_prd} (prd)")

show_e2_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=e2_company_id,
)
)
e2_p, e2_r, e2_a, e2_prd = show_e2_accounts_response.balances
print(f"Account balances of E2: {e2_p} (p), {e2_r} (r), {e2_a} (a), {e2_prd} (prd)")

show_a1_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=a1_company_id,
)
)
a1_p, a1_r, a1_a, a1_prd = show_a1_accounts_response.balances
print(f"Account balances of A1: {a1_p} (p), {a1_r} (r), {a1_a} (a), {a1_prd} (prd)")

show_a2_accounts_response = show_my_accounts_use_case(
    request=show_my_accounts.ShowMyAccountsRequest(
        current_user=a2_company_id,
)
)
a2_p, a2_r, a2_a, a2_prd = show_a2_accounts_response.balances
print(f"Account balances of A2: {a2_p} (p), {round(a2_r, 2)} (r), {a2_a} (a), {a2_prd} (prd)")

The workers notice, that the planning has been quite accurate, but that they have spent more euros than planned (E2 prd=+10,50, E1 prd=+1,40).

It shows also in the global app statistics, that accounts have been overdrawn:

In [ ]:
get_statistics_response = get_statistics_use_case()
print(f"Available certificates and hours on 'A' accounts: {round(get_statistics_response.certificates_count, 2)}")
print(f"Available product: {get_statistics_response.available_product}")